# Extracting correct Arxiv ID and version from title

This code is executed in google collab

In [10]:
pip install -q arxiv openreview openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.3 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# Standard library imports
import os
import re
import ast
import json
import time
import tqdm
import random
import getpass
import pandas as pd
import urllib.parse
import urllib.request
import xml.etree.ElementTree as ET
from collections import defaultdict
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from concurrent.futures import ThreadPoolExecutor, as_completed

# Third-party imports
import openai
import requests
import feedparser
import openreview
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

openai.api_type = "azure"
openai.api_base = "XXX"
openai.api_version = "XXX"
openai.api_key = "XXX"
deployment_name =  "XXX"

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:115.0) Gecko/20100101 Firefox/115.0",
    "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:112.0) Gecko/20100101 Firefox/112.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Linux; Android 13; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.141 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 12; SM-G991B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPad; CPU OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.199 Safari/537.36 OPR/106.0.0.0",
    "Mozilla/5.0 (Linux; Android 12; SAMSUNG SM-G991U) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/21.0 Chrome/109.0.0.0 Mobile Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.89 YaBrowser/23.9.1.754 Yowser/2.5 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.139 Safari/537.36 Vivaldi/6.5.3206.47",
]

In [3]:
def download_file(url, save_path, retries=3, delay=250):
  """Not Used"""
  if not url:
      return
  if url.startswith("/"):
      url = "https://openreview.net" + url

  for attempt in range(1, retries + 1):
      try:
          r = requests.get(url, stream=True)
          if r.status_code == 200:
              with open(save_path, 'wb') as f:
                  for chunk in r.iter_content(chunk_size=8192):
                      f.write(chunk)
              print(f"Downloaded successfully: {url}")
              return
          else:
              print(f"Attempt {attempt}: Failed to download {url}. Status code: {r.status_code}")
      except Exception as e:
          print(f"Attempt {attempt}: Error downloading {url}: {e}")

      if attempt < retries:
          print(f"Retrying in {delay} seconds...")
          time.sleep(delay)

  print(f"Failed to download {url} after {retries} attempts.")

def normalize(text):
    return ' '.join(text.strip().split()).lower()

def fetch_arxiv_api(url, user_agents, max_retries=2, retry_delay=250):
  for attempt in range(1, max_retries + 1):
    try:
      headers = {'User-Agent': random.choice(user_agents)}
      req = Request(url, headers=headers)
      with urlopen(req) as data:
        return data.read().decode('utf-8')
    except (URLError, HTTPError) as e:
      print(f"[Attempt {attempt}] Failed to fetch URL: {e}")
      if attempt < max_retries:
        print(f"Retrying in {retry_delay // 60} minutes...")
        time.sleep(retry_delay)
      else:
        return None

def jaccard_similarity(a, b):
    def tokenize(text):
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        return set(text.split())

    set_a = tokenize(a)
    set_b = tokenize(b)
    intersection = set_a & set_b
    union = set_a | set_b
    return len(intersection) / len(union) if union else 0.0

---
Start here

In [20]:
base_dir = "/content/drive/MyDrive/LLMs/openreview/ICLR2025/rejected"
csv_file = "papers_list"
df = pd.read_csv(f"{base_dir}/{csv_file}.csv")
print(len(df))

4949


In [21]:
arxiv_list = [None] * len(df)
arxiv_info_list = [None] * len(df)

def process_paper(index, paper):
  try:
    title = paper['title']
    paper_id = paper['paperid']
    paper_dir = os.path.join(base_dir, paper_id)
    encoded_text = urllib.parse.quote_plus(title)
    url = f'http://export.arxiv.org/api/query?search_query=all:{encoded_text}&start=0&max_results=5'
    response = fetch_arxiv_api(url, user_agents)
    root = ET.fromstring(response)
    ns = {'atom': 'http://www.w3.org/2005/Atom'}

    best_score = 0
    arxiv_id_url = ""
    arxiv_id = ""
    version = ""
    arxiv_title = ""
    published = ""
    updated = ""

    for entry in root.findall('atom:entry', ns):
      current_title = entry.find('atom:title', ns).text.strip()
      sim_score = jaccard_similarity(current_title, title)

      if normalize(current_title) == normalize(title):
          sim_score = 1.0

      if sim_score > best_score:
          id_url = entry.find('atom:id', ns).text
          match = re.search(r'arxiv.org/abs/(\d+\.\d+)(v\d+)', id_url)
          if match:
              best_score = sim_score
              arxiv_id_url = id_url
              arxiv_id = match.group(1)
              version = match.group(2)
              arxiv_title = current_title
              published = entry.find('atom:published', ns).text
              updated = entry.find('atom:updated', ns).text

    if best_score < 0.88 or not arxiv_id:
      print("********** PAPER NOT FOUND **********")
      print("arxiv_title: ", arxiv_title)
      print("title: ", title)
      return index, "", {}

    url = f'https://arxiv.org/abs/{arxiv_id}'
    max_retries = 3
    retry_delay = 2
    response = None
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            break
        except Exception:
            if attempt < max_retries - 1:
                print(f"Error fetching {url}, attempt {attempt + 1} of {max_retries}. Retrying...")
                time.sleep(retry_delay)

    if response is None:
      return index, arxiv_id, {}

    soup = BeautifulSoup(response.text, 'html.parser')
    history_div = soup.find('div', class_='submission-history')

    if not history_div:
      return index, arxiv_id, {}

    history_text = history_div.get_text(separator="\n").strip()
    prompt = f"""
    You are an assistant that parses arXiv submission history blocks and extracts version timestamps.
    Given the following raw submission history, return a JSON dictionary where each key is the version
    (e.g., "v1") and the value is the submission timestamp in ISO 8601 format (UTC, ending with Z).

    Text:
    \"\"\"
    {history_text}
    \"\"\"

    Output JSON:
    """

    response = openai.ChatCompletion.create(
        engine=deployment_name,
        messages=[
            {"role": "system", "content": "You extract structured data from raw text."},
            {"role": "user", "content": prompt}
        ]
    )

    output_json = response["choices"][0]["message"]["content"]
    output_json = ast.literal_eval(output_json)

    # pdf_url = f"https://arxiv.org/pdf/{arxiv_id}{suffix}.pdf"
    # file_name = f"arxiv_{paper_id}_{suffix}.pdf"
    # file_path = os.path.join(paper_dir, file_name)
    # try:
    #   download_file(pdf_url, file_path)
    # except Exception:
    #   print(f"Failed to download {pdf_url}")

    # src_url = f"https://arxiv.org/src/{arxiv_id}{suffix}"
    # file_name = f"arxiv_latex_{paper_id}_{suffix}.zip"
    # file_path = os.path.join(paper_dir, file_name)
    # try:
    #     download_file(src_url, file_path)
    # except Exception:
    #     print(f"Failed to download {src_url}")
    print(output_json)
    return index, arxiv_id, output_json

  except Exception as e:
    print(f"Error processing paper {paper_id}: {e}")
    return index, "", {}

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(process_paper, idx, row) for idx, row in df.iterrows()]
    for future in tqdm.tqdm(as_completed(futures), total=len(futures), desc="Processing submissions"):
        idx, arxiv_id, version_info = future.result()
        arxiv_list[idx] = arxiv_id
        arxiv_info_list[idx] = version_info

df['arxiv_id'] = arxiv_list
df['arxiv_info'] = arxiv_info_list

********** PAPER NOT FOUND **********
arxiv_title:  
title:  WISDOM: Progressive Curriculum Synthesis Makes LLMs Better Mathematical Reasoner


Processing submissions:   0%|          | 0/4949 [00:00<?, ?it/s]

********** PAPER NOT FOUND **********
arxiv_title:  Identifying rote learning and the supporting effects of hints in drills
title:  Rethinking Memorization in LLMs: On Learning by Rote vs. with Understanding


Processing submissions:   0%|          | 3/4949 [00:00<05:52, 14.04it/s]

********** PAPER NOT FOUND **********
arxiv_title:  
title:  FEABench: Evaluating Language Models on Real World Physics Reasoning Ability
********** PAPER NOT FOUND **********
arxiv_title:  LLaMA-MoE v2: Exploring Sparsity of LLaMA from Perspective of
  Mixture-of-Experts with Post-Training
title:  Smoothness Bridges Sparsity and Stability in MoEs


...

********** PAPER NOT FOUND **********
arxiv_title:  In-context Reinforcement Learning with Algorithm Distillation
title:  Memory-Efficient Algorithm Distillation for In-context Reinforcement Learning
********** PAPER NOT FOUND **********
arxiv_title:  
title:  CIRCUIT: A Benchmark for Circuit Interpretation and Reasoning Capabilities of LLMs


Processing submissions: 100%|█████████▉| 4948/4949 [30:35<00:00,  3.67it/s]

********** PAPER NOT FOUND **********
arxiv_title:  Accelerating LLM Inference with Lossless Speculative Decoding Algorithms
  for Heterogeneous Vocabularies
title:  A Unified Framework for Speculative Decoding with Multiple Drafters as a Bandit


Processing submissions: 100%|██████████| 4949/4949 [30:37<00:00,  2.69it/s]

{'v1': '2024-10-17T17:56:01Z', 'v2': '2024-12-04T17:03:13Z'}


In [19]:
global_csv_path = os.path.join(base_dir, f"{csv_file}.csv")
df.to_csv(global_csv_path, index=False)
# print("Test ok?")